In [1]:
import json
import os, statistics, math

def extract_step_to_acc(path: str):
    """
    Read a raw metrics file and return {step: test_acc} from even-numbered lines only.
    - 1-based line numbering (keep only even lines)
    - JSON parse; keep only split == 'test'
    - Map: step -> acc (latest occurrence wins if duplicated)
    """
    result = {}
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, start=1):  # 1-based
            line = line.strip()
            if not line or (i % 2 != 0):  # skip odd lines
                continue
            try:
                obj = json.loads(line)
            except json.JSONDecodeError:
                continue
            if obj.get("split") != "test":
                continue
            step = obj.get("step")
            acc = obj.get("acc")
            if step is not None and acc is not None:
                result[step] = acc  # latest one wins
    return result

In [2]:
def summarize_by_step(root='.', start=1, end=53, prefix='client_', ext='.raw',
                      variance='population', step_keys=None, require_all=False):
    """
    Aggregate across clients PER STEP and return TWO dictionaries:
      - mean_by_step: {step: mean_test_acc_over_clients}
      - var_by_step:  {step: variance_test_acc_over_clients}

    Parameters:
      - variance: 'population' (statistics.pvariance) | 'sample' (statistics.variance)
      - step_keys: optional iterable of steps to enforce (e.g., [0,10,20,...,100]).
                   If None, uses the union of steps observed across clients.
      - require_all: if True, only include a step if ALL clients have a value for it.
                     If False, compute from available values.

    Notes:
      - Uses extract_step_to_acc(path) from earlier cell (even-line, split=='test').
      - Missing files or missing steps are ignored per 'require_all' policy.
    """
    # Collect per-step lists of accuracies across clients
    accs_by_step = {}
    client_count = 0
    for i in range(start, end + 1):
        path = os.path.join(root, f"{prefix}{i:03d}{ext}")
        try:
            d = extract_step_to_acc(path)
        except FileNotFoundError:
            # Missing client file; skip
            continue
        client_count += 1
        for step, acc in d.items():
            accs_by_step.setdefault(step, []).append(acc)

    # Decide which steps to include
    if step_keys is None:
        steps = sorted(accs_by_step.keys())
    else:
        steps = list(step_keys)

    mean_by_step = {}
    var_by_step  = {}
    for step in steps:
        vals = accs_by_step.get(step, [])
        if not vals:
            continue  # no data at this step
        if require_all and len(vals) != client_count:
            # Skip this step because not all clients provided it
            continue
        m = sum(vals) / len(vals)
        if variance == 'population':
            v = statistics.pvariance(vals)
        elif variance == 'sample':
            v = statistics.variance(vals) if len(vals) > 1 else float('nan')
        else:
            raise ValueError("variance must be 'population' or 'sample'")
        mean_by_step[step] = m
        var_by_step[step]  = v
    return mean_by_step, var_by_step

In [3]:
mean_dict, var_dict = summarize_by_step(root='.', start=1, end=53)
print(mean_dict)
print(var_dict)

{0: 0.5641509433962263, 10: 0.5608490566037736, 20: 0.5580188679245283, 30: 0.5617924528301886, 40: 0.5693396226415094, 50: 0.569811320754717, 60: 0.5712264150943397, 70: 0.569811320754717, 80: 0.5627358490566038, 90: 0.5712264150943396, 100: 0.5707547169811321}
{0: 0.006309184763260947, 10: 0.004728996084015663, 20: 0.005820131719473122, 30: 0.005981221075115699, 40: 0.0061000355998576, 50: 0.0075556247775008896, 60: 0.006660288358846564, 70: 0.005456568173727305, 80: 0.00595808116767533, 90: 0.005881986472054112, 100: 0.00473433606265575}
